In [1]:
%load_ext autoreload 
%autoreload 2
%matplotlib inline
import numpy as np
import sympy
import scipy.sparse.linalg
import scipy.sparse as sps
from math import pi

import setup_grids
from core.constit import second_order_tensor
from core.bc import bc
from fvdiscr import mpfa, fvutils
from viz import cell_data

In [2]:
# Homogeneous media

###
# This is where parameters can be modified to alter the convergence test.
# The remaining lines should 
np.random.seed(42)
base = 4
domain = np.array([1, 1])
basedim = np.array([base, base])
num_refs = 5
grid_type = 'cart'
pert = 0.5
### End of parameter definitions

# Permeability tensor, scalar for simplicity
perm = 1

# Analytical solution
x, y = sympy.symbols('x y')
u = sympy.sin(x) * sympy.cos(y)
u_f = sympy.lambdify((x, y), u, 'numpy')
dux = sympy.diff(u, x)
duy = sympy.diff(u, y)
dux_f = sympy.lambdify((x, y), dux, 'numpy')
duy_f = sympy.lambdify((x, y), duy, 'numpy')
rhs = -sympy.diff(dux, x) - sympy.diff(duy, y)
rhs_f = sympy.lambdify((x, y), rhs, 'numpy')


def run_convergence(grid_type):
    u_err = []
    flux_err = []

    for g in setup_grids.grid_sequence(basedim, num_refs, grid_type, pert):
        # Reset the random seed for every grid realization.
        # This should make no difference for the convergence test, 
        # but it makes sure that we can run unit tests based on the values obtained
        # here.
        np.random.seed(42)
        
        # Permeability tensor
        k = second_order_tensor.SecondOrderTensor(2, perm * np.ones(g.num_cells))

        # Set type of boundary conditions - Dirichlet
        bound_faces = g.get_boundary_faces()
        bound_cond = bc.BoundaryCondition(g, bound_faces, ['dir'] * bound_faces.size)
        
        # MPFA discretization, and system matrix
        flux, bound_flux = mpfa.mpfa(g, k, bound_cond)
        div = fvutils.scalar_divergence(g)
        a = div * flux
        
        # Boundary conditions
        xf = g.face_centers
        u_bound = np.zeros(g.num_faces)
        u_bound[bound_faces] = u_f(xf[0, bound_faces], xf[1, bound_faces])
        
        # Right hand side - contribution from the solution and the boundary conditions
        xc = g.cell_centers
        b = rhs_f(xc[0], xc[1]) * g.cell_volumes + div * bound_flux * u_bound

        # Solve system, derive fluxes
        u_num = scipy.sparse.linalg.spsolve(a, b)
        flux_num = flux * u_num - bound_flux * u_bound

        # Exact solution
        u_ex = u_f(xc[0], xc[1])
        du_ex_faces = np.vstack((dux_f(xf[0], xf[1]), duy_f(xf[0], xf[1])))
        flux_ex = -np.sum(g.face_normals[:2] * du_ex_faces, axis=0)
        flux_diff = flux_num - flux_ex
  
        u_err.append(np.sqrt(np.sum(g.cell_volumes * (u_num - u_ex)**2)) /
                     np.sqrt(np.sum(g.cell_volumes * u_ex**2)))
        flux_err.append(np.sqrt(np.sum((g.face_areas ** g.dim) * flux_diff**2))/
                        np.sqrt(np.sum((g.face_areas ** g.dim) * flux_ex**2)))
    return u_err, flux_err

grids = ['cart', 'triangular']

for gr in grids:
    u, f = run_convergence(gr)
    print('Grid: ', gr)
    print(u,'pressure error')
    print(f,'flux error')



Grid:  cart
[0.0073686732581183341, 0.0020614141524652875, 0.00052069850436366801, 0.00012689494678806644, 3.3174376428851243e-05] pressure error
[0.018364051591446429, 0.0069607346238991594, 0.0018965063877406545, 0.00076139949877853633, 0.00034071679478220134] flux error
Grid:  triangular
[0.0026129292546810826, 0.00060836749272092218, 0.00016721778412009586, 3.8951673047906369e-05, 9.5183746497811669e-06] pressure error
[0.011939673878238723, 0.0059878440468308525, 0.0019404259450671838, 0.00080962523721565024, 0.00037383233647136536] flux error


In [10]:
# Neumann homogeneous media

###
# This is where parameters can be modified to alter the convergence test.
# The remaining lines should 
np.random.seed(42)
base = 4
domain = np.array([1, 1])
basedim = np.array([base, base])
num_refs = 5
grid_type = 'cart'
pert = 0
### End of parameter definitions

# Permeability tensor, scalar for simplicity
perm = 1

# Analytical solution
x, y = sympy.symbols('x y')
u = sympy.sin(x) * sympy.cos(y)
u_f = sympy.lambdify((x, y), u, 'numpy')
dux = sympy.diff(u, x)
duy = sympy.diff(u, y)
dux_f = sympy.lambdify((x, y), dux, 'numpy')
duy_f = sympy.lambdify((x, y), duy, 'numpy')
rhs = -sympy.diff(dux, x) - sympy.diff(duy, y)
rhs_f = sympy.lambdify((x, y), rhs, 'numpy')


def run_convergence(grid_type):
    u_err = []
    flux_err = []

    for g in setup_grids.grid_sequence(basedim, num_refs, grid_type, pert):
        # Reset the random seed for every grid realization.
        # This should make no difference for the convergence test, 
        # but it makes sure that we can run unit tests based on the values obtained
        # here.
        np.random.seed(42)
        
        # Permeability tensor
        k = second_order_tensor.SecondOrderTensor(2, perm * np.ones(g.num_cells))
        
        # Exact solution
        xf = g.face_centers
        xc = g.cell_centers
        u_ex = u_f(xc[0], xc[1])
        du_ex_faces = np.vstack((dux_f(xf[0], xf[1]), duy_f(xf[0], xf[1])))
        flux_ex = -np.sum(g.face_normals[:2] * du_ex_faces, axis=0)
        # Set type of boundary conditions - Dirichlet
        n = np.max(g.nodes)
        top = np.ravel(np.argwhere(g.face_centers[1, :] > n - 1e-10))
        bot = np.ravel(np.argwhere(g.face_centers[1, :] < 1e-10))
        left = np.ravel(np.argwhere(g.face_centers[0, :] < 1e-10))
        right = np.ravel(np.argwhere(g.face_centers[0, :] > n - 1e-10))

        dir_faces = top
        neu_faces = np.hstack((left,right,bot))
        bound_cond = bc.BoundaryCondition(g, dir_faces, ['dir'] * dir_faces.size)
        
        # MPFA discretization, and system matrix
        flux, bound_flux = mpfa.mpfa(g, k, bound_cond)
        div = fvutils.scalar_divergence(g)
        a = div * flux
        
        # Boundary conditions
        nfi, _, sgn = sps.find(g.cell_faces[neu_faces,:])
        u_bound = np.zeros(g.num_faces)
        u_bound[dir_faces] = u_f(xf[0, dir_faces], xf[1, dir_faces])
        # innflow is always positive, so need to flip flux according to normal direction.
        u_bound[neu_faces[nfi]] = flux_ex[neu_faces[nfi]] * (- sgn)
        
        # Right hand side - contribution from the solution and the boundary conditions
        b = rhs_f(xc[0], xc[1]) * g.cell_volumes + div * bound_flux * u_bound

        # Solve system, derive fluxes
        u_num = scipy.sparse.linalg.spsolve(a, b)
        flux_num = flux * u_num - bound_flux * u_bound

        # Error estimates
        flux_diff = flux_num - flux_ex
        u_err.append(np.sqrt(np.sum(g.cell_volumes * (u_num - u_ex)**2)) /
                     np.sqrt(np.sum(g.cell_volumes * u_ex**2)))
        flux_err.append(np.sqrt(np.sum((g.face_areas ** g.dim) * flux_diff**2))/
                        np.sqrt(np.sum((g.face_areas ** g.dim) * flux_ex**2)))
    return u_err, flux_err

grids = ['cart', 'triangular']

for gr in grids:
    u, f = run_convergence(gr)
    print('Grid: ', gr)
    print(u,'pressure error')
    print(f,'flux error')


Grid:  cart
[0.0072022282476458871, 0.0017874530851249273, 0.0004460699832864939, 0.00011146844452597047, 2.7864055519112392e-05] pressure error
[0.0022799310895371587, 0.0005807385581614021, 0.00014548172425695252, 3.6307175629253304e-05, 9.0606912100608403e-06] flux error
Grid:  triangular
[0.0031972554177003968, 0.00077313127744489706, 0.00019036873350168699, 4.7223465626264777e-05, 1.1758251902283734e-05] pressure error
[0.0031608355124965325, 0.00093419113467097762, 0.00031344628635151199, 0.00011077735929434521, 3.9597222269329265e-05] flux error


In [4]:
# Heterogeneous media

###
# This is where parameters can be modified to alter the convergence test.
# The remaining lines should stay unaltered
np.random.seed(42)
base = 4
domain = np.array([1, 1])
basedim = np.array([base, base])
num_refs = 3
grid_typ = 'cart'
pert = 0.5

kappa_list = [1e-6, 1, 1e6]

### End of parameter definitions

# Permeability tensor, scalar for simplicity
perm = 1

def chi(x, y):
    return np.logical_and(np.greater(x, 0.5), np.greater(y, 0.5))

# Analytical solution
x, y = sympy.symbols('x y')
u = sympy.sin(2 * x * pi) * sympy.sin(2 * y * pi) 
u_f = sympy.lambdify((x, y), u, 'numpy')
dux = sympy.diff(u, x)
duy = sympy.diff(u, y)
dux_f = sympy.lambdify((x, y), dux, 'numpy')
duy_f = sympy.lambdify((x, y), duy, 'numpy')
rhs = -sympy.diff(dux, x) - sympy.diff(duy, y)
rhs_f = sympy.lambdify((x, y), rhs, 'numpy')


def run_convergence(grid_type):
    u_err = np.zeros((num_refs, len(kappa_list)))
    flux_err = np.copy(u_err)
    
    for iter1, g in enumerate(setup_grids.grid_sequence_fixed_lines(basedim, num_refs, grid_type, pert, subdom_func=chi)):
        # Reset the random seed for every grid realization.
        # This should make no difference for the convergence test, 
        # but it makes sure that we can run unit tests based on the values obtained
        # here.
        np.random.seed(42)
        
        for iter2, kappa in enumerate(kappa_list):
            char_func_cells = chi(g.cell_centers[0], g.cell_centers[1]) * 1.
            perm_vec = (1 - char_func_cells) + kappa * char_func_cells
            perm = second_order_tensor.SecondOrderTensor(2, perm_vec)

            bound_faces = g.get_boundary_faces()
            bound_cond = bc.BoundaryCondition(g, bound_faces, ['dir'] * bound_faces.size)
            flux, bound_flux = mpfa.mpfa(g, perm, bound_cond, inverter='python')

            xc = g.cell_centers
            xf = g.face_centers
            char_func_bound = chi(xf[0, bound_faces], xf[1, bound_faces]) * 1
            
            u_bound = np.zeros(g.num_faces)
            u_bound[bound_faces] = u_f(xf[0, bound_faces], xf[1, bound_faces]) \
                                / ((1 - char_func_bound) + kappa * char_func_bound)

            div = fvutils.scalar_divergence(g)
            a = div * flux

            b = rhs_f(xc[0], xc[1]) * g.cell_volumes + div * bound_flux * u_bound

            u_num = scipy.sparse.linalg.spsolve(a, b)
            u_ex = u_f(xc[0], xc[1]) / ((1 - char_func_cells) + kappa * char_func_cells)
            
            flux_num = flux * u_num - bound_flux * u_bound
            du_ex_faces = np.vstack((dux_f(xf[0], xf[1]), duy_f(xf[0], xf[1])))
            flux_ex = -np.sum(g.face_normals[:2] * du_ex_faces, axis=0)
            flux_diff = flux_num - flux_ex

            u_err[iter1, iter2] = (np.sqrt(np.sum(g.cell_volumes * (u_num - u_ex)**2)) /
                         np.sqrt(np.sum(g.cell_volumes * u_ex**2)))
            flux_err[iter1, iter2] = (np.sqrt(np.sum((g.face_areas ** g.dim) * flux_diff**2))/
                            np.sqrt(np.sum((g.face_areas ** g.dim) * flux_ex**2)))
    return u_err, flux_err

grids = ['cart', 'triangular']

for gr in grids:
    u, f = run_convergence(gr)
    print('Grid: ', gr)
    print(u,'pressure error')
    print(f,'flux error')



Grid:  cart
[[ 0.28144397  0.3116494   0.28115807]
 [ 0.08134885  0.0741203   0.06868011]
 [ 0.01484212  0.01673688  0.01728718]] pressure error
[[ 0.18205415  0.1865744   0.20400765]
 [ 0.05365452  0.05195488  0.05255201]
 [ 0.01689975  0.01675983  0.01677493]] flux error
Grid:  triangular
[[ 0.0909746   0.18646187  0.14360106]
 [ 0.0305652   0.03734376  0.03559147]
 [ 0.00709532  0.00900442  0.00857817]] pressure error
[[ 0.13965683  0.13292462  0.13615825]
 [ 0.04759549  0.04591041  0.04656731]
 [ 0.01712045  0.01694357  0.01697797]] flux error


In [5]:
print(dux_f(0.5, 0.7))

5.97566432948
